In [1]:
from hyperopt import * #probably do not need to import everything
# import findspark
# findspark.init()
# import pyspark [TODO]
import pickle
import time
import numpy as np
import pandas as pd
import os
import csv
import statistics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GroupShuffleSplit
from sklearn.multioutput import MultiOutputClassifier


from sklearn.svm import LinearSVC

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneGroupOut

from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import GroupKFold

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In [393]:
pd.set_option('display.max_columns', None)

In [394]:
import configparser

def read_ini(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    dataset_csv = config["hyperopt"]["dataset_csv"]
    X_column_range = config["hyperopt"]["X_column_range"]
    Y_column_names = config["hyperopt"]["Y_column_names"].split(" ")
    utteranceID = config["hyperopt"]["utteranceID"]
    Group = config["hyperopt"]["Group_column"]
    output_name = config["hyperopt"]["output_name"]
    resultsDir = config["hyperopt"]["resultsDir"]
    guide_metric_name = config["hyperopt"]["guide_metric"]
    return dataset_csv, X_column_range, Y_column_names,\
        utteranceID, Group, output_name, resultsDir, guide_metric_name
 
dataset_csv, X_column_range, Y_column_names, \
utteranceID, Group, output_name, resultsDir, guide_metric_name  = read_ini("config_3.ini")
df_dataset = pd.read_csv(dataset_csv)
df_dataset.dtypes


utteranceID                              object
Group                                    object
CPS_CONST_SharesU_Situation             float64
CPS_CONST_SharesU_CorrectSolutions      float64
CPS_CONST_SharesU_IncorrectSolutions    float64
                                         ...   
bert_507                                float64
bert_508                                float64
bert_509                                float64
bert_510                                float64
bert_511                                float64
Length: 533, dtype: object

In [395]:
df_dataset

,utteranceID,Group,CPS_CONST_SharesU_Situation,CPS_CONST_SharesU_CorrectSolutions,CPS_CONST_SharesU_IncorrectSolutions,CPS_CONST_EstablishesCG_Confirms,CPS_CONST_EstablishesCG_Interrupts,CPS_NEG_Responds_Reasons,CPS_NEG_Responds_QuestionsOthers,CPS_NEG_Responds_Responds,CPS_NEG_MonitorsE_Results,CPS_NEG_MonitorsE_Strategizes,CPS_NEG_MonitorsE_Save,CPS_NEG_MonitorsE_GivingUp,CPS_MAINTAIN_Initiative_Suggestions,CPS_MAINTAIN_Initiative_Compliments,CPS_MAINTAIN_Initiative_Criticizes,CPS_MAINTAIN_FulfillsR_Support,CPS_MAINTAIN_FulfillsR_Apologizes,CPS_MAINTAIN_FulfillsR_InitiatesOffTopic,CPS_MAINTAIN_FulfillsR_JoinsOffTopic,bert_0,bert_1,bert_2,bert_3,bert_4,bert_5,bert_6,bert_7,bert_8,bert_9,bert_10,bert_11,bert_12,bert_13,bert_14,bert_15,bert_16,bert_17,bert_18,bert_19,bert_20,bert_21,bert_22,bert_23,bert_24,bert_25,bert_26,bert_27,bert_28,bert_29,bert_30,bert_31,bert_32,bert_33,bert_34,bert_35,bert_36,bert_37,bert_38,bert_39,bert_40,bert_41,bert_42,bert_43,bert_44,bert_45,bert_46,bert_47,bert_48,bert_49,bert_50,bert_51,bert_52,bert_53,bert_54,bert_55,bert_56,bert_57,bert_58,bert_59,bert_60,bert_61,bert_62,bert_63,bert_64,bert_65,bert_66,bert_67,bert_68,bert_69,bert_70,bert_71,bert_72,bert_73,bert_74,bert_75,bert_76,bert_77,bert_78,bert_79,bert_80,bert_81,bert_82,bert_83,bert_84,bert_85,bert_86,bert_87,bert_88,bert_89,bert_90,bert_91,bert_92,bert_93,bert_94,bert_95,bert_96,bert_97,bert_98,bert_99,bert_100,bert_101,bert_102,bert_103,bert_104,bert_105,bert_106,bert_107,bert_108,bert_109,bert_110,bert_111,bert_112,bert_113,bert_114,bert_115,bert_116,bert_117,bert_118,bert_119,bert_120,bert_121,bert_122,bert_123,bert_124,bert_125,bert_126,bert_127,bert_128,bert_129,bert_130,bert_131,bert_132,bert_133,bert_134,bert_135,bert_136,bert_137,bert_138,bert_139,bert_140,bert_141,bert_142,bert_143,bert_144,bert_145,bert_146,bert_147,bert_148,bert_149,bert_150,bert_151,bert_152,bert_153,bert_154,bert_155,bert_156,bert_157,bert_158,bert_159,bert_160,bert_161,bert_162,bert_163,bert_164,bert_165,bert_166,bert_167,bert_168,bert_169,bert_170,bert_171,bert_172,bert_173,bert_174,bert_175,bert_176,bert_177,bert_178,bert_179,bert_180,bert_181,bert_182,bert_183,bert_184,bert_185,bert_186,bert_187,bert_188,bert_189,bert_190,bert_191,bert_192,bert_193,bert_194,bert_195,bert_196,bert_197,bert_198,bert_199,bert_200,bert_201,bert_202,bert_203,bert_204,bert_205,bert_206,bert_207,bert_208,bert_209,bert_210,bert_211,bert_212,bert_213,bert_214,bert_215,bert_216,bert_217,bert_218,bert_219,bert_220,bert_221,bert_222,bert_223,bert_224,bert_225,bert_226,bert_227,bert_228,bert_229,bert_230,bert_231,bert_232,bert_233,bert_234,bert_235,bert_236,bert_237,bert_238,bert_239,bert_240,bert_241,bert_242,bert_243,bert_244,bert_245,bert_246,bert_247,bert_248,bert_249,bert_250,bert_251,bert_252,bert_253,bert_254,bert_255,bert_256,bert_257,bert_258,bert_259,bert_260,bert_261,bert_262,bert_263,bert_264,bert_265,bert_266,bert_267,bert_268,bert_269,bert_270,bert_271,bert_272,bert_273,bert_274,bert_275,bert_276,bert_277,bert_278,bert_279,bert_280,bert_281,bert_282,bert_283,bert_284,bert_285,bert_286,bert_287,bert_288,bert_289,bert_290,bert_291,bert_292,bert_293,bert_294,bert_295,bert_296,bert_297,bert_298,bert_299,bert_300,bert_301,bert_302,bert_303,bert_304,bert_305,bert_306,bert_307,bert_308,bert_309,bert_310,bert_311,bert_312,bert_313,bert_314,bert_315,bert_316,bert_317,bert_318,bert_319,bert_320,bert_321,bert_322,bert_323,bert_324,bert_325,bert_326,bert_327,bert_328,bert_329,bert_330,bert_331,bert_332,bert_333,bert_334,bert_335,bert_336,bert_337,bert_338,bert_339,bert_340,bert_341,bert_342,bert_343,bert_344,bert_345,bert_346,bert_347,bert_348,bert_349,bert_350,bert_351,bert_352,bert_353,bert_354,bert_355,bert_356,bert_357,bert_358,bert_359,bert_360,bert_361,bert_362,bert_363,bert_364,bert_365,bert_366,bert_367,bert_368,bert_369,bert_370,bert_371,bert_372,bert_373,bert_374,bert_375,bert_376,bert_377,bert_378,bert_379,bert_380,bert_381,bert_382,bert_383,bert_384,bert_385,bert_386,

In [396]:
df_dataset.describe(include='all')

,utteranceID,Group,CPS_CONST_SharesU_Situation,CPS_CONST_SharesU_CorrectSolutions,CPS_CONST_SharesU_IncorrectSolutions,CPS_CONST_EstablishesCG_Confirms,CPS_CONST_EstablishesCG_Interrupts,CPS_NEG_Responds_Reasons,CPS_NEG_Responds_QuestionsOthers,CPS_NEG_Responds_Responds,CPS_NEG_MonitorsE_Results,CPS_NEG_MonitorsE_Strategizes,CPS_NEG_MonitorsE_Save,CPS_NEG_MonitorsE_GivingUp,CPS_MAINTAIN_Initiative_Suggestions,CPS_MAINTAIN_Initiative_Compliments,CPS_MAINTAIN_Initiative_Criticizes,CPS_MAINTAIN_FulfillsR_Support,CPS_MAINTAIN_FulfillsR_Apologizes,CPS_MAINTAIN_FulfillsR_InitiatesOffTopic,CPS_MAINTAIN_FulfillsR_JoinsOffTopic,bert_0,bert_1,bert_2,bert_3,bert_4,bert_5,bert_6,bert_7,bert_8,bert_9,bert_10,bert_11,bert_12,bert_13,bert_14,bert_15,bert_16,bert_17,bert_18,bert_19,bert_20,bert_21,bert_22,bert_23,bert_24,bert_25,bert_26,bert_27,bert_28,bert_29,bert_30,bert_31,bert_32,bert_33,bert_34,bert_35,bert_36,bert_37,bert_38,bert_39,bert_40,bert_41,bert_42,bert_43,bert_44,bert_45,bert_46,bert_47,bert_48,bert_49,bert_50,bert_51,bert_52,bert_53,bert_54,bert_55,bert_56,bert_57,bert_58,bert_59,bert_60,bert_61,bert_62,bert_63,bert_64,bert_65,bert_66,bert_67,bert_68,bert_69,bert_70,bert_71,bert_72,bert_73,bert_74,bert_75,bert_76,bert_77,bert_78,bert_79,bert_80,bert_81,bert_82,bert_83,bert_84,bert_85,bert_86,bert_87,bert_88,bert_89,bert_90,bert_91,bert_92,bert_93,bert_94,bert_95,bert_96,bert_97,bert_98,bert_99,bert_100,bert_101,bert_102,bert_103,bert_104,bert_105,bert_106,bert_107,bert_108,bert_109,bert_110,bert_111,bert_112,bert_113,bert_114,bert_115,bert_116,bert_117,bert_118,bert_119,bert_120,bert_121,bert_122,bert_123,bert_124,bert_125,bert_126,bert_127,bert_128,bert_129,bert_130,bert_131,bert_132,bert_133,bert_134,bert_135,bert_136,bert_137,bert_138,bert_139,bert_140,bert_141,bert_142,bert_143,bert_144,bert_145,bert_146,bert_147,bert_148,bert_149,bert_150,bert_151,bert_152,bert_153,bert_154,bert_155,bert_156,bert_157,bert_158,bert_159,bert_160,bert_161,bert_162,bert_163,bert_164,bert_165,bert_166,bert_167,bert_168,bert_169,bert_170,bert_171,bert_172,bert_173,bert_174,bert_175,bert_176,bert_177,bert_178,bert_179,bert_180,bert_181,bert_182,bert_183,bert_184,bert_185,bert_186,bert_187,bert_188,bert_189,bert_190,bert_191,bert_192,bert_193,bert_194,bert_195,bert_196,bert_197,bert_198,bert_199,bert_200,bert_201,bert_202,bert_203,bert_204,bert_205,bert_206,bert_207,bert_208,bert_209,bert_210,bert_211,bert_212,bert_213,bert_214,bert_215,bert_216,bert_217,bert_218,bert_219,bert_220,bert_221,bert_222,bert_223,bert_224,bert_225,bert_226,bert_227,bert_228,bert_229,bert_230,bert_231,bert_232,bert_233,bert_234,bert_235,bert_236,bert_237,bert_238,bert_239,bert_240,bert_241,bert_242,bert_243,bert_244,bert_245,bert_246,bert_247,bert_248,bert_249,bert_250,bert_251,bert_252,bert_253,bert_254,bert_255,bert_256,bert_257,bert_258,bert_259,bert_260,bert_261,bert_262,bert_263,bert_264,bert_265,bert_266,bert_267,bert_268,bert_269,bert_270,bert_271,bert_272,bert_273,bert_274,bert_275,bert_276,bert_277,bert_278,bert_279,bert_280,bert_281,bert_282,bert_283,bert_284,bert_285,bert_286,bert_287,bert_288,bert_289,bert_290,bert_291,bert_292,bert_293,bert_294,bert_295,bert_296,bert_297,bert_298,bert_299,bert_300,bert_301,bert_302,bert_303,bert_304,bert_305,bert_306,bert_307,bert_308,bert_309,bert_310,bert_311,bert_312,bert_313,bert_314,bert_315,bert_316,bert_317,bert_318,bert_319,bert_320,bert_321,bert_322,bert_323,bert_324,bert_325,bert_326,bert_327,bert_328,bert_329,bert_330,bert_331,bert_332,bert_333,bert_334,bert_335,bert_336,bert_337,bert_338,bert_339,bert_340,bert_341,bert_342,bert_343,bert_344,bert_345,bert_346,bert_347,bert_348,bert_349,bert_350,bert_351,bert_352,bert_353,bert_354,bert_355,bert_356,bert_357,bert_358,bert_359,bert_360,bert_361,bert_362,bert_363,bert_364,bert_365,bert_366,bert_367,bert_368,bert_369,bert_370,bert_371,bert_372,bert_373,bert_374,bert_375,bert_376,bert_377,bert_378,bert_379,bert_380,bert_381,bert_382,bert_383,bert_384,bert_385,bert_386,

In [397]:
X_start, X_end = X_column_range.split(":")
if(len(X_end) == 0): 
    X = df_dataset.iloc[:, int(X_start):]
else:
    X = df_dataset.iloc[:, int(X_start):int(X_end)]
print(X.columns)
X = X.to_numpy()

Index(['bert_0', 'bert_1', 'bert_2', 'bert_3', 'bert_4', 'bert_5', 'bert_6',
       'bert_7', 'bert_8', 'bert_9',
       ...
       'bert_502', 'bert_503', 'bert_504', 'bert_505', 'bert_506', 'bert_507',
       'bert_508', 'bert_509', 'bert_510', 'bert_511'],
      dtype='object', length=512)


In [398]:
Y = df_dataset[Y_column_names].copy(deep=True)
num_classes = len(Y_column_names)
Y = Y.fillna(0)
Y = Y.to_numpy()
Y = Y.flatten()
print(Y_column_names)
print(Y[:100])

['CPS_CONST_EstablishesCG_Interrupts']
[0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0.]


In [399]:
utteranceIDs = pd.DataFrame(df_dataset[utteranceID], columns=[utteranceID])
utteranceIDs = utteranceIDs.to_numpy()
utteranceIDs=utteranceIDs.reshape(-1)

In [400]:
groups = pd.DataFrame(df_dataset[Group], columns=[Group])
groups = groups.to_numpy()
groups = groups.reshape(-1)
unique_groups = np.unique(groups)
print(unique_groups)

['Group_01' 'Group_02' 'Group_03' 'Group_04' 'Group_05' 'Group_06'
 'Group_07' 'Group_08' 'Group_09' 'Group_10']


In [401]:
print (X.shape, Y.shape)

(1822, 512) (1822,)


In [402]:
if not os.path.exists(resultsDir):
        os.mkdir(resultsDir)
modelcsv =os.path.join(resultsDir, output_name)
if not os.path.exists(modelcsv):
        os.mkdir(modelcsv)
else:
        print("WARNING: DIRECTORY ALREADY EXISTS")

In [403]:
cv = LeaveOneGroupOut()
best = 0
results = []

In [404]:
space = hp.choice('classifier_type', [
    # {
    #     'type': 'AdaBoost',
    #     'n_estimators': hp.choice('AdaBoost_n_estimators', list(range(30, 100, 11))),
    #     # 'learning_rate': hp.choice('AdaBoost_learning_rate', list(np.linspace(0.1, 4.1, 50))),
    #     'learning_rate': hp.choice('AdaBoost_learning_rate', [.001, 0.1, 1.0, 10]),
    #     'algorithm': hp.choice('AdaBoost_algorithm', ['SAMME', 'SAMME.R'])
    # },
    
    # {
    #     'type': 'naive_bayes',
        
    # },
    # {
    #     'type': 'logistic_regression',
    #     'penalty' : hp.choice('logistic_regression_penalty', ['l2', 'none']),
    #     'solver' : hp.choice('logistic_regression_solver', ['lbfgs', 'newton-cg', 'sag', 'saga'])
    # },
    # {
    #     'type': 'LinSVC',
    #     'C': hp.uniform('LinSVC.C', 0, 3.0),

    # },
    # {
    #     'type': 'SVC',
    #     'C': hp.choice('C', [0.1, 1, 10, 100, 1000]),
    #     'gamma': hp.choice('gamma', [1, 0.1, 0.01, 0.001, 0.0001]),
    #     'kernel': hp.choice('kernal', ['rbf', 'sigmoid']),

    # },
    
    # {
    #     'type': 'decision_tree',
        
    #     'criterion': hp.choice('decision_tree.criterion', ["gini", "entropy"]),
    #     'splitter': hp.choice('splitter', ["best", "random"]),
    #     'max_features': hp.choice('decision_tree.max_features', ["auto", "sqrt", "log2"]),

    # },
    {
        'type': 'randomforest',
        'n_estimators': hp.choice('n_estimators', list(range(20, 170, 16))),
        'criterion': hp.choice('criterion', ["gini", "entropy"]),
        'max_features': hp.choice('max_features', [None, "sqrt", "log2"]),
        
    # },
    # {
    #     'type': 'knn',
    #     'n_neighbors': hp.choice('knn_n_neighbors', list(range(1, 150, 10)))
    }
    
])

In [405]:
def hyperopt_train_test(params, groups, cv):
    resultsDict = {}
    model_name = params['type']
    predictioncsv = model_name
    del params['type']
    for item in params.values():
        predictioncsv += '_' + str(item)
    predictioncsv += '.csv'
    resultsDict["predictioncsv"] = predictioncsv
    
    if model_name == 'naive_bayes':
        clf = GaussianNB(**params)
    elif model_name == 'AdaBoost':
        clf = AdaBoostClassifier(**params)
    elif model_name == 'LinSVC':
        clf = LinearSVC(**params)      
    elif model_name == 'decision_tree':
        clf = DecisionTreeClassifier(**params, random_state=1) 
    elif model_name == 'randomforest':
        clf = RandomForestClassifier(**params, random_state=1)
    elif model_name == 'knn':
        clf = KNeighborsClassifier(**params)
    elif model_name == 'logistic_regression':
        clf = LogisticRegression(**params, random_state=1)
    elif model_name == 'SVC':
        clf = SVC(**params, probability=True)    
    else:
        return 0
    
    resultsDict["model_name"] = model_name
    
    # y_pred = cross_val_predict(clf, X, Y, groups=groups, n_jobs=20,cv=cv, verbose=0, method='predict')
    # estimator = 'predict_proba'
    # if model_name == 'LinSVC':
    #     y_score = cross_val_predict(clf, X, Y, groups=groups, n_jobs=20,cv=cv, verbose=0, method='decision_function')
    # else:
    #     y_score = cross_val_predict(clf, X, Y, groups=groups, n_jobs=20,cv=cv, verbose=0, method=estimator)
    #     y_score = np.array(y_score[:, 1])

    # y_pred = np.array(y_pred)
    # print(Y.shape)
    # print(y_pred.shape)
    # print(y_score.shape)

    # resultsDict["y_pred"] = y_pred
    # resultsDict["f1"] = f1_score(Y, y_pred)
    # resultsDict["accuracy"] = accuracy_score(Y, y_pred)
    # try:
    #     resultsDict["auroc"] = roc_auc_score(Y, y_score)
    #     print(resultsDict["auroc"])
    # except:
    #     print(f"ROC_AUC Issue with class")
    #     resultsDict["auroc"] = -1
    # try:
    #     resultsDict["kappa"] = cohen_kappa_score(Y, y_pred)
    # except:
    #     print(f"Kappa issue with class")
    #     resultsDict["kappa"] = -1
    # resultsDict["matthews_cc"] = matthews_corrcoef(Y, y_pred)
    # resultsDict["confusion_matrix"] = multilabel_confusion_matrix(Y, y_pred)
    # return resultsDict[guide_metric_name], resultsDict

    scoring = {
    'f1': make_scorer(f1_score, average='weighted'),  # You can adjust 'average' as needed
    'kappa': make_scorer(cohen_kappa_score),
    'accuracy': make_scorer(accuracy_score)
    }

    listOfScores = []
    scores = cross_validate(clf, X, Y,groups=groups, n_jobs=20,cv=cv,scoring=scoring)
    print("Mean F1 Score:", scores['test_f1'].mean())
    print("Mean Kappa Score:", scores['test_kappa'].mean())
    print("Mean Accuracy Score:", scores['test_accuracy'].mean())
    resultsDict["averageF1"] = scores['test_f1'].mean()
    resultsDict["averageKappa"] = scores['test_kappa'].mean()
    resultsDict["averageAccuracy"] = scores['test_accuracy'].mean()
    resultsDict["stdF1"] = scores['test_f1'].std()
    resultsDict["stdKappa"] = scores['test_kappa'].std()
    resultsDict["stdAccuracy"] = scores['test_accuracy'].std()
    listOfScores.append(scores)
    return resultsDict[guide_metric_name], resultsDict 

    
    

In [406]:
count = 1
def f(params):
    global best, count, avg
    count += 1

    guide_metric, scores = hyperopt_train_test(params.copy(), groups, cv)
    results.append(scores)
    # with open(os.path.join(modelcsv,scores["predictioncsv"]), 'w', newline = '') as modelCSV:
    #     writer = csv.writer(modelCSV)
    #     print("model: ", scores["predictioncsv"])
    #     writer.writerow(['utteranceID', 'True Label', 'Pred Label'])
    #     for pred in range(len(scores["y_pred"])):
    #         writer.writerow([utteranceIDs[pred], Y[pred], scores["y_pred"][pred]])
    if guide_metric > best:
        print (scores)
        best = guide_metric
    else: 
        print (scores)
        
    return {'loss': -guide_metric, 'status': STATUS_OK, 'eval_time': time.time(), 
            'other_stuff': {'model':scores["model_name"],'f1_0':scores["averageF1"] ,'accuracy': scores["averageAccuracy"],
                           'kappa':scores["averageKappa"]}
    }


In [407]:
best = fmin(
            f, 
            space, 
            algo=tpe.suggest, 
            max_evals= 50,)
print ('best:' )
print(best)
print(space_eval(space, best))
df = pd.DataFrame.from_dict(results)
df = df.sort_values(by=guide_metric_name, ascending=False)
df.to_csv(os.path.join(modelcsv, output_name) + ".csv")

(1822,)                                                
(1822,)                                                
(1822,)                                                
0.7251415037098675                                     
model:                                                 
AdaBoost_SAMME.R_1.0_96.csv                            
{'predictioncsv': 'AdaBoost_SAMME.R_1.0_96.csv', 'model_name': 'AdaBoost', 'y_pred': array([0., 0., 0., ..., 0., 0., 0.]), 'f1': 0.3481481481481482, 'accuracy': 0.8068057080131723, 'auroc': 0.7251415037098675, 'kappa': 0.2368513741206485, 'matthews_cc': 0.23946803581226594, 'confusion_matrix': array([[[  94,  210],
        [ 142, 1376]],

       [[1376,  142],
        [ 210,   94]]], dtype=int64)}
(1822,)                                                                             
(1822,)                                                                             
(1822,)                                                                             
0.803784